# Introduction to Data Science 2025

# Week 4

In this week's exercise, we look at prompting and zero- and few-shot task settings. Below is a text generation example from https://github.com/TurkuNLP/intro-to-nlp/blob/master/text_generation_pipeline_example.ipynb demonstrating how to load a text generation pipeline with a pre-trained model and generate text with a given prompt. Your task is to load a similar pre-trained generative model and assess whether the model succeeds at a set of tasks in zero-shot, one-shot, and two-shot settings.

**Note: Downloading and running the pre-trained model locally may take some time. Alternatively, you can open and run this notebook on [Google Colab](https://colab.research.google.com/), as assumed in the following example.**

## Text generation example

This is a brief example of how to run text generation with a causal language model and `pipeline`.

Install [transformers](https://huggingface.co/docs/transformers/index) python package. This will be used to load the model and tokenizer and to run generation.

In [1]:
!pip install --quiet transformers

Import the `AutoTokenizer`, `AutoModelForCausalLM`, and `pipeline` classes. The first two support loading tokenizers and generative models from the [Hugging Face repository](https://huggingface.co/models), and the last wraps a tokenizer and a model for convenience.

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

/home/kazan/Desktop/IntroToDS/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load a generative model and its tokenizer. You can substitute any other generative model name here (e.g. [other TurkuNLP GPT-3 models](https://huggingface.co/models?sort=downloads&search=turkunlp%2Fgpt3)), but note that Colab may have issues running larger models. 

In [2]:
MODEL_NAME = 'TurkuNLP/gpt3-finnish-large'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

Instantiate a text generation pipeline using the tokenizer and model.

In [3]:
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device=model.device
)

Device set to use cpu


We can now call the pipeline with a text prompt; it will take care of tokenizing, encoding, generation, and decoding:

In [4]:
output = pipe('Terve, miten menee?', max_new_tokens=25)

print(output)

[{'generated_text': 'Terve, miten menee?”\n”Huonosti”, vastasin ja toivoin, ettei hän sanoisi ”huonosti”.\n”Näytät siltä kuin olisit'}]


Just print the text

In [24]:
print(output[0]['generated_text'])

Terve, miten menee?”
”Hyvin, kiitos.”
”Kiva kuulla.”
”Kuule, minulla on sinulle asiaa.”



We can also call the pipeline with any arguments that the model `generate` function supports. For details on text generation using `transformers`, see e.g. [this tutorial](https://huggingface.co/blog/how-to-generate).

Example with sampling and a high `temperature` parameter to generate more chaotic output:

In [8]:
output = pipe(
    'Terve, miten menee?',
    do_sample=False,
    temperature=10.0,
    max_new_tokens=25
)

print(output[0]['generated_text'])

Terve, miten menee?”
”Hyvin, kiitos.”
”Kiva kuulla.”
”Kuule, minulla on sinulle asiaa.”



## Exercise 1

Your task is to assess whether a generative model succeeds in the following tasks in zero-shot, one-shot, and two-shot settings:

- binary sentiment classification (positive / negative)

- person name recognition

- two-digit addition (e.g. 11 + 22 = 33)

For example, for assessing whether a generative model can name capital cities, we could use the following prompts:

- zero-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""
- one-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Sweden?\
	>Answer: Stockholm
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""
- two-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Sweden?\
	>Answer: Stockholm
	>
	>Question: What is the capital of Denmark?\
	>Answer: Copenhagen
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""

You can do the tasks either in English or Finnish and use a generative model of your choice from the Hugging Face models repository, for example the following models:

- English: `gpt2-large`
- Finnish: `TurkuNLP/gpt3-finnish-large`

You can either come up with your own instructions for the tasks or use the following:

- English:
	- binary sentiment classification: "Do the following texts express a positive or negative sentiment?"
	- person name recognition: "List the person names occurring in the following texts."
	- two-digit addition: "This is a first grade math exam."
- Finnish:
	- binary sentiment classification: "Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?"
	- person name recognition: "Listaa seuraavissa teksteissä mainitut henkilönnimet."
	- two-digit addition: "Tämä on ensimmäisen luokan matematiikan koe."

Come up with at least two test cases for each of the three tasks, and come up with your own one- and two-shot examples.

In [10]:
# Use this cell for your code

def run_case(prompt, max_new_tokens=20):
    output = pipe(
        prompt,
        do_sample=False,   # deterministic
        temperature=None,
        top_k=None,
        top_p=None,
        max_new_tokens=max_new_tokens
    )
    return output[0]['generated_text']


# --- Task 1: Sentiment classification ---
prompt1 = """Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?

Teksti: Rakastin tätä elokuvaa, se sai minut hymyilemään.
Vastaus:
"""
print(f"Sentiment zeroshot:\n{run_case(prompt1)}\n")

prompt2 = """Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?

Teksti: Pizza oli herkullista ja palvelu erinomaista.
Vastaus: positiivinen

Teksti: Tämä oli ajanhukkaa, niin tylsää.
Vastaus:
"""
print(f"Sentiment oneshot:\n{run_case(prompt2)}\n")

prompt3 = """Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?

Teksti: Pizza oli herkullista ja palvelu erinomaista.
Vastaus: positiivinen

Teksti: Ruoka oli kylmää ja tarjoilija epäystävällinen.
Vastaus: negatiivinen

Teksti: Tämä oli ajanhukkaa, niin tylsää.
Vastaus:
"""
print(f"Sentiment twoshot:\n{run_case(prompt3)}\n")


# --- Task 2: Person name recognition ---
prompt4 = """Listaa seuraavissa teksteissä mainitut henkilönnimet.

Teksti: Maria ja Joonas menivät puistoon.
Vastaus:
"""
print(f"Names zeroshot:\n{run_case(prompt4)}\n")

prompt5 = """Listaa seuraavissa teksteissä mainitut henkilönnimet.

Teksti: Liisa ja Matti ovat ystäviä.
Vastaus: Liisa, Matti

Teksti: Maria ja Joonas menivät puistoon.
Vastaus:
"""
print(f"Names oneshot:\n{run_case(prompt5)}\n")

prompt6 = """Listaa seuraavissa teksteissä mainitut henkilönnimet.

Teksti: Liisa ja Matti ovat ystäviä.
Vastaus: Liisa, Matti

Teksti: Kalle Virtanen tapasi Antti Heikkisen.
Vastaus: Kalle Virtanen, Antti Heikkinen

Teksti: Maria ja Joonas menivät puistoon.
Vastaus:
"""
print(f"Names twoshot:\n{run_case(prompt6)}\n")


# --- Task 3: Two-digit addition ---
prompt7 = """Tämä on ensimmäisen luokan matematiikan koe.

Kysymys: 12 + 34 = ?
Vastaus:
"""
print(f"Math zeroshot:\n{run_case(prompt7)}\n")

prompt8 = """Tämä on ensimmäisen luokan matematiikan koe.

Kysymys: 11 + 22 = ?
Vastaus: 33

Kysymys: 12 + 34 = ?
Vastaus:
"""
print(f"Math oneshot:\n{run_case(prompt8)}\n")

prompt9 = """Tämä on ensimmäisen luokan matematiikan koe.

Kysymys: 11 + 22 = ?
Vastaus: 33

Kysymys: 40 + 15 = ?
Vastaus: 55

Kysymys: 12 + 34 = ?
Vastaus:
"""
print(f"Math twoshot:\n{run_case(prompt9)}\n")


Sentiment zeroshot:
Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?

Teksti: Rakastin tätä elokuvaa, se sai minut hymyilemään.
Vastaus:
Elokuva on hyvä, jos se saa sinut hymyilemään. Jos se saa sinut tuntemaan surua, se on huono

Sentiment oneshot:
Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?

Teksti: Pizza oli herkullista ja palvelu erinomaista.
Vastaus: positiivinen

Teksti: Tämä oli ajanhukkaa, niin tylsää.
Vastaus:
negatiivinen

Teksti: Tämä oli hyvä, mutta en halua enää koskaan nähdä tätä paikkaa.
Vastaus:
negatiiv

Sentiment twoshot:
Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?

Teksti: Pizza oli herkullista ja palvelu erinomaista.
Vastaus: positiivinen

Teksti: Ruoka oli kylmää ja tarjoilija epäystävällinen.
Vastaus: negatiivinen

Teksti: Tämä oli ajanhukkaa, niin tylsää.
Vastaus:
negatiivinen

Teksti: Ruoka oli hyvää ja palvelu ystävällistä.
Vastaus:
negatiivinen

Teksti: Ruoka oli hyvää

Names zeroshot:

**Submit this exercise by submitting your code and your answers to the above questions as comments on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**